In [1]:
from openpyxl import load_workbook
import pandas as pd
import openpyxl as xl; 
import functools
from math import pi
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_Rows', 13000)
pd.set_option('display.max_columns', 12000)
import warnings
warnings.filterwarnings('ignore')

# **All_Data**

In [2]:
row_data= pd.read_csv('Aug_new_7.csv')

In [3]:
#row_data.sort_values("UUID", inplace = True) 
#row_data.drop_duplicates(subset ="UUID", inplace = True)

In [4]:
row_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199162 entries, 0 to 2199161
Data columns (total 21 columns):
COHORT                        object
CLIENT                        object
NEW_RECURRING                 object
CREATEDAT                     object
SCORING_MODEL                 object
SCORE                         float64
LEAD_DATE_EXCL_AUTOREJECTS    object
LEADD                         float64
LAST_OFFER_DATE               object
OFFER                         float64
REJECT_DATE                   object
REJECTD                       float64
OFFER_ACCEPTED_DATE           object
OFFER_ACCEPTED                float64
PAYOUT_DATE                   object
PAYOUT                        float64
CUSTOMER_TYPE_DETAILED        object
APPLICATION_ID                object
DPD_EOM                       float64
MONTHS_SINCE_PAYOUT           float64
UUID                          object
dtypes: float64(8), object(13)
memory usage: 352.3+ MB


In [5]:
row_data.CLIENT.unique()

array(['MONEDONOW_PL', 'MONEDONOW_ES', 'KREDITO_24_ES', 'ZAIMO',
       'MONEDO_FINANCE_PL', 'MONEDO_BNPL_PL', 'MONEDO_FINANCE_IN'],
      dtype=object)

# 1. spain

## 1.a spain Micro Loans (KREDITO_24_ES)

In [6]:
Espain_ML = row_data.loc[row_data.CLIENT =='KREDITO_24_ES']
bins= [0,0.65,0.67,0.70,0.74,1]
labels = ['b_0-0.65','c_0.65-0.67','d_0.67-0.70','e_0.70-0.74','f_0.74-1']
Espain_ML['band'] = pd.cut(Espain_ML['SCORE'], bins=bins, labels=labels, right=False)
Espain_ML.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
4,2020-02,KREDITO_24_ES,NEW,2020-02-12 21:37:48.356000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 21:41:09.490000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS157593R11,NaN
10,2020-02,KREDITO_24_ES,NEW,2020-02-12 17:56:08.022000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 18:34:15.434000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS156872R11,NaN
13,2020-02,KREDITO_24_ES,NEW,2020-02-12 22:14:35.763000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS157646R11,NaN
17,2020-02,KREDITO_24_ES,NEW,2020-02-12 09:46:20.658000,ES_K24_MODEL,0.63281,2020-02-12 09:51:33.757000,1.0,NaN,NaN,2020-02-12 09:51:41.287000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS155501R11,b_0-0.65
18,2020-02,KREDITO_24_ES,NEW,2020-02-12 11:08:28.218000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 11:11:37.325000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS155588R11,NaN


In [7]:
Espain_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179468 entries, 4 to 2198559
Data columns (total 22 columns):
COHORT                        179468 non-null object
CLIENT                        179468 non-null object
NEW_RECURRING                 179468 non-null object
CREATEDAT                     179468 non-null object
SCORING_MODEL                 41870 non-null object
SCORE                         41870 non-null float64
LEAD_DATE_EXCL_AUTOREJECTS    41868 non-null object
LEADD                         41868 non-null float64
LAST_OFFER_DATE               6005 non-null object
OFFER                         6005 non-null float64
REJECT_DATE                   95213 non-null object
REJECTD                       95213 non-null float64
OFFER_ACCEPTED_DATE           5744 non-null object
OFFER_ACCEPTED                5744 non-null float64
PAYOUT_DATE                   4481 non-null object
PAYOUT                        4481 non-null float64
CUSTOMER_TYPE_DETAILED        0 non-null object
APPL

## LEADD-New

In [8]:
Lead_ES_K24 = Espain_ML.loc[(Espain_ML['LEADD'] == 1) & (Espain_ML['NEW_RECURRING'] =='NEW') & (Espain_ML['SCORING_MODEL'] == 'ES-K24-HIGH-NEW-PRED-ML-m585939-2')]
Lead_ES_k24_outPut= pd.pivot_table(Lead_ES_K24, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
Lead_ES_k24_outPut_x = Lead_ES_k24_outPut.copy()
Lead_ES_k24_outPut_x.columns = [''.join(col).strip() for col in Lead_ES_k24_outPut.columns.values]
Lead_ES_k24_outPut_y=Lead_ES_k24_outPut_x.sort_values(by=['band'],ascending = False)
Lead_ES_k24_outPut_y

,LEADD2020-04,LEADD2020-05,LEADD2020-06,LEADD2020-07
band,,,,
f_0.74-1,45,208,192,108
e_0.70-0.74,25,84,86,46
d_0.67-0.70,26,82,61,37
c_0.65-0.67,10,52,46,27
b_0-0.65,1507,5687,5170,3474


## OFFER-New

In [9]:
offer_ES_K24 = Espain_ML.loc[(Espain_ML['OFFER'] == 1) & (Espain_ML['NEW_RECURRING'] =='NEW') & (Espain_ML['SCORING_MODEL'] == 'ES-K24-HIGH-NEW-PRED-ML-m585939-2')]
offer_ES_K24_outPut= pd.pivot_table(offer_ES_K24, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
offer_ES_K24_outPut_x = offer_ES_K24_outPut.copy()
offer_ES_K24_outPut_x.columns = [''.join(col).strip() for col in offer_ES_K24_outPut.columns.values]
offer_ES_K24_outPut_y=offer_ES_K24_outPut_x.sort_values(by=['band'],ascending = False)

## PAYOUT-New

In [10]:
PAYOUT_ES_K24 = Espain_ML.loc[(Espain_ML['PAYOUT'] == 1) & (Espain_ML['NEW_RECURRING'] =='NEW') & (Espain_ML['SCORING_MODEL'] == 'ES-K24-HIGH-NEW-PRED-ML-m585939-2')]
PAYOUT_ES_K24_outPut= pd.pivot_table(PAYOUT_ES_K24, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
PAYOUT_ES_K24_outPut_x = PAYOUT_ES_K24_outPut.copy()
PAYOUT_ES_K24_outPut_x.columns = [''.join(col).strip() for col in PAYOUT_ES_K24_outPut.columns.values]
PAYOUT_ES_K24_outPut_y=PAYOUT_ES_K24_outPut_x.sort_values(by=['band'],ascending = False)
PAYOUT_ES_K24_outPut_y

,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,
f_0.74-1,26,112,115,61
e_0.70-0.74,11,44,40,25
d_0.67-0.70,15,41,26,18
c_0.65-0.67,3,10,9,6
b_0-0.65,1,2,3,4


# 30+ DPD

In [11]:
Dpd_ES_K24 = Espain_ML.loc[(Espain_ML['LEADD'] == 1) & (Espain_ML['NEW_RECURRING'] =='NEW') & (Espain_ML['SCORING_MODEL'] == 'ES-K24-HIGH-NEW-PRED-ML-m585939-2') & (Espain_ML['MONTHS_SINCE_PAYOUT'] == 2) & (Espain_ML['DPD_EOM'] >= 30)]
Dpd_ES_K24_outPut= pd.pivot_table(Dpd_ES_K24, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_ES_K24_outPut.columns = [''.join(col).strip() for col in Dpd_ES_K24_outPut.columns.values]
Dpd_ES_K24_outPut_y=Dpd_ES_K24_outPut.sort_values(by=['band'],ascending = False)
divid_es_ML= Dpd_ES_K24_outPut_y/PAYOUT_ES_K24_outPut_y
divid_es_ML_y=divid_es_ML.sort_values(by=['band'],ascending = False)
divid_es_ML_y

,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,
f_0.74-1,0.423077,0.205357,0.069565,NaN
e_0.70-0.74,0.454545,0.272727,0.075000,NaN
d_0.67-0.70,0.133333,0.341463,0.153846,NaN
c_0.65-0.67,0.000000,0.200000,0.000000,NaN
b_0-0.65,1.000000,0.000000,0.000000,NaN


# Spain_ML_REC_data

In [12]:
Espain_ML_rec = row_data.loc[row_data.CLIENT =='KREDITO_24_ES']
bins= [0,0.53,0.62,0.72,0.82,1]
labels = ['b_0-0.53','c_0.53-0.62','d_0.62-0.72','e_0.72-0.82','f_0.82-1']
Espain_ML_rec['band'] = pd.cut(Espain_ML_rec['SCORE'], bins=bins, labels=labels, right=False)
Espain_ML_rec.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
4,2020-02,KREDITO_24_ES,NEW,2020-02-12 21:37:48.356000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 21:41:09.490000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS157593R11,NaN
10,2020-02,KREDITO_24_ES,NEW,2020-02-12 17:56:08.022000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 18:34:15.434000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS156872R11,NaN
13,2020-02,KREDITO_24_ES,NEW,2020-02-12 22:14:35.763000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS157646R11,NaN
17,2020-02,KREDITO_24_ES,NEW,2020-02-12 09:46:20.658000,ES_K24_MODEL,0.63281,2020-02-12 09:51:33.757000,1.0,NaN,NaN,2020-02-12 09:51:41.287000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS155501R11,d_0.62-0.72
18,2020-02,KREDITO_24_ES,NEW,2020-02-12 11:08:28.218000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 11:11:37.325000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLS155588R11,NaN


## LEADD-Rec

In [13]:
Lead_ES_K24_Rec = Espain_ML_rec.loc[(Espain_ML_rec['LEADD'] == 1) & (Espain_ML_rec['NEW_RECURRING'] !='NEW') & (Espain_ML_rec['SCORING_MODEL'] == 'ES-K24-HIGH-NEW-PRED-ML-m585939-2')]
Lead_ES_K24_Rec_outPut= pd.pivot_table(Lead_ES_K24_Rec, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
Lead_ES_K24_Rec_outPut_x = Lead_ES_K24_Rec_outPut.copy()
Lead_ES_K24_Rec_outPut_x.columns = [''.join(col).strip() for col in Lead_ES_K24_Rec_outPut_x.columns.values]
Lead_ES_K24_Rec_outPut_y=Lead_ES_K24_Rec_outPut_x.sort_values(by=['band'],ascending = False)

## OFFER-Rec

In [14]:
offer_ES_K24_rec = Espain_ML_rec.loc[(Espain_ML_rec['OFFER'] == 1) & (Espain_ML_rec['NEW_RECURRING'] !='NEW') & (Espain_ML_rec['SCORING_MODEL'] == 'ES-K24-HIGH-NEW-PRED-ML-m585939-2')]
offer_ES_K24_rec_outPut= pd.pivot_table(offer_ES_K24_rec, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
offer_ES_K24_rec_outPut_x = offer_ES_K24_rec_outPut.copy()
offer_ES_K24_rec_outPut_x.columns = [''.join(col).strip() for col in offer_ES_K24_rec_outPut.columns.values]
offer_ES_K24_rec_outPut_y= offer_ES_K24_rec_outPut_x.sort_values(by=['band'],ascending = False)

## PAYOUT-Rec

In [15]:
PAYOUT_ES_K24_rec = Espain_ML_rec.loc[(Espain_ML_rec['PAYOUT'] == 1) & (Espain_ML_rec['NEW_RECURRING'] !='NEW') & (Espain_ML_rec['SCORING_MODEL'] == 'ES-K24-HIGH-NEW-PRED-ML-m585939-2')]
PAYOUT_ES_K24_rec_outPut = pd.pivot_table(PAYOUT_ES_K24_rec, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
PAYOUT_ES_K24_rec_outPut_x = PAYOUT_ES_K24_rec_outPut.copy()
PAYOUT_ES_K24_rec_outPut_x.columns = [''.join(col).strip() for col in PAYOUT_ES_K24_rec_outPut.columns.values]
PAYOUT_ES_K24_rec_outPut_y= PAYOUT_ES_K24_rec_outPut_x.sort_values(by=['band'],ascending = False)
PAYOUT_ES_K24_rec_outPut_y

,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,
f_0.82-1,22,75,96,87
e_0.72-0.82,19,87,102,111
d_0.62-0.72,21,83,111,126
c_0.53-0.62,18,88,97,112
b_0-0.53,16,78,104,95


# DPD30+

In [16]:
Dpd_ES_IL = Espain_ML_rec.loc[(Espain_ML_rec['PAYOUT'] == 1) & (Espain_ML_rec['NEW_RECURRING'] !='NEW') & (Espain_ML_rec['SCORING_MODEL'] == 'ES-K24-HIGH-NEW-PRED-ML-m585939-2') & (Espain_ML_rec['MONTHS_SINCE_PAYOUT'] == 2) & (Espain_ML['DPD_EOM'] >= 30)]
Dpd_ES_IL_outPut= pd.pivot_table(Dpd_ES_IL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_ES_IL_outPut.columns = [''.join(col).strip() for col in Dpd_ES_IL_outPut.columns.values]
Dpd_ES_IL_outPut_x= Dpd_ES_IL_outPut.sort_values(by=['band'],ascending = False)
divid_es_IL= Dpd_ES_IL_outPut_x/PAYOUT_ES_K24_rec_outPut_y
divid_es_ml_rec_y=divid_es_IL.sort_values(by=['band'],ascending = False)
divid_es_ml_rec_y

,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,
f_0.82-1,0.000000,0.066667,0.020833,NaN
e_0.72-0.82,0.105263,0.103448,0.068627,NaN
d_0.62-0.72,0.095238,0.192771,0.108108,NaN
c_0.53-0.62,0.388889,0.136364,0.103093,NaN
b_0-0.53,0.125000,0.166667,0.076923,NaN


# 1.b NAP - Spain Installment Loan

In [17]:
MONEDONOW_ES = row_data.loc[row_data.CLIENT =='MONEDONOW_ES']
bins= [0,0.65,0.68,0.71,0.73,1]
labels = ['b_0-0.65','c_0.65-0.68','d_0.68-0.71','e_0.71-0.73','f_0.73-1']
MONEDONOW_ES['band'] = pd.cut(MONEDONOW_ES['SCORE'], bins=bins, labels=labels, right=False)
MONEDONOW_ES.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
1,2020-02,MONEDONOW_ES,NEW,2020-02-12 12:50:34.720000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 12:57:00.072000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DLS539497R11,NaN
2,2020-02,MONEDONOW_ES,NEW,2020-02-12 10:08:30.726000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DLS538778R11,NaN
5,2020-02,MONEDONOW_ES,NEW,2020-02-12 14:17:03.975000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DLS540112R11,NaN
8,2020-02,MONEDONOW_ES,NEW,2020-02-12 13:02:10.237000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 13:08:34.399000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DLS539606R11,NaN
9,2020-02,MONEDONOW_ES,NEW,2020-02-12 18:28:17.181000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DLS541094R11,NaN


In [18]:
MONEDONOW_ES.drop_duplicates(subset ="UUID", keep = 'first', inplace = True)
MONEDONOW_ES.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 202852 entries, 1 to 2199158
Data columns (total 22 columns):
COHORT                        202852 non-null object
CLIENT                        202852 non-null object
NEW_RECURRING                 202852 non-null object
CREATEDAT                     202852 non-null object
SCORING_MODEL                 37787 non-null object
SCORE                         37787 non-null float64
LEAD_DATE_EXCL_AUTOREJECTS    41261 non-null object
LEADD                         41261 non-null float64
LAST_OFFER_DATE               1977 non-null object
OFFER                         1977 non-null float64
REJECT_DATE                   93295 non-null object
REJECTD                       93295 non-null float64
OFFER_ACCEPTED_DATE           1569 non-null object
OFFER_ACCEPTED                1569 non-null float64
PAYOUT_DATE                   1027 non-null object
PAYOUT                        1027 non-null float64
CUSTOMER_TYPE_DETAILED        41261 non-null object


## LEADD

In [19]:
Lead_MONEDONOW_ES = MONEDONOW_ES.loc[(MONEDONOW_ES['LEADD'] == 1) &  (MONEDONOW_ES['NEW_RECURRING'] =='NEW') & ((MONEDONOW_ES['SCORING_MODEL'] == 'ES-MNOW-HIGH-NEW-PRED-KK_CB-ST-m1bfe24') 
                                 | (MONEDONOW_ES['SCORING_MODEL'] =='ES-MNOW-HIGH-NEW-PRED-KK_CB-ST-m1bfe24-1'))]

In [20]:
Lead_MONEDONOW_ES_outPut= pd.pivot_table(Lead_MONEDONOW_ES, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
Lead_MONEDONOW_ES_outPut.sort_values(by=['band'],ascending = False)
Lead_MONEDONOW_ES_outPut_x = Lead_MONEDONOW_ES_outPut.copy()
Lead_MONEDONOW_ES_outPut_x.columns = [''.join(col).strip() for col in Lead_MONEDONOW_ES_outPut.columns.values]
Lead_MONEDONOW_ES_outPut_y=Lead_MONEDONOW_ES_outPut_x.sort_values(by=['band'],ascending = False)
Lead_MONEDONOW_ES_outPut_y

,LEADD2020-02,LEADD2020-03,LEADD2020-04
band,,,
f_0.73-1,122,155,67
e_0.71-0.73,85,74,26
d_0.68-0.71,187,166,94
c_0.65-0.68,240,191,95
b_0-0.65,13949,14607,6471


## OFFER

In [21]:
OFFER_MONEDONOW_ES = MONEDONOW_ES.loc[(MONEDONOW_ES['OFFER'] == 1) & ((MONEDONOW_ES['SCORING_MODEL'] == 'ES-MNOW-HIGH-NEW-PRED-KK_CB-ST-m1bfe24') 
                                     | (MONEDONOW_ES['SCORING_MODEL'] =='ES-MNOW-HIGH-NEW-PRED-KK_CB-ST-m1bfe24-1'))]
OFFER_MONEDONOW_ES_outPut= pd.pivot_table(OFFER_MONEDONOW_ES, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
OFFER_MONEDONOW_ES_outPut_x = OFFER_MONEDONOW_ES_outPut.copy()
OFFER_MONEDONOW_ES_outPut_x.columns = [''.join(col).strip() for col in OFFER_MONEDONOW_ES_outPut.columns.values]
OFFER_MONEDONOW_ES_outPut_y=OFFER_MONEDONOW_ES_outPut_x.sort_values(by=['band'],ascending = False)


## PAYOUT

In [22]:
PAYOUT_MONEDONOW_ES = MONEDONOW_ES.loc[(MONEDONOW_ES['PAYOUT'] == 1) & ((MONEDONOW_ES['SCORING_MODEL'] == 'ES-MNOW-HIGH-NEW-PRED-KK_CB-ST-m1bfe24') | (MONEDONOW_ES['SCORING_MODEL'] =='ES-MNOW-HIGH-NEW-PRED-KK_CB-ST-m1bfe24-1'))]
PAYOUT_MONEDONOW_ES_outPut= pd.pivot_table(PAYOUT_MONEDONOW_ES, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
PAYOUT_MONEDONOW_ES_outPut_x = PAYOUT_MONEDONOW_ES_outPut.copy()
PAYOUT_MONEDONOW_ES_outPut_x.columns = [''.join(col).strip() for col in PAYOUT_MONEDONOW_ES_outPut.columns.values]
PAYOUT_MONEDONOW_ES_outPut_y=PAYOUT_MONEDONOW_ES_outPut_x.sort_values(by=['band'],ascending = False)


# DPD30+

In [23]:
Dpd_ES_MONEDONOW_ES_IL =  PAYOUT_MONEDONOW_ES.loc[(PAYOUT_MONEDONOW_ES['DPD_EOM'] >= 30) & (PAYOUT_MONEDONOW_ES['MONTHS_SINCE_PAYOUT'] == 3)]
Dpd_ES_MONEDONOW_ES_IL_outPut= pd.pivot_table(Dpd_ES_MONEDONOW_ES_IL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_ES_MONEDONOW_ES_IL_outPut.columns = [''.join(col).strip() for col in Dpd_ES_MONEDONOW_ES_IL_outPut.columns.values]
Dpd_ES_MONEDONOW_ES_IL_outPut_x= Dpd_ES_MONEDONOW_ES_IL_outPut.sort_values(by=['band'],ascending = False)
divid_es_MONEDONOW_IL= Dpd_ES_MONEDONOW_ES_IL_outPut_x/PAYOUT_MONEDONOW_ES_outPut_y
divid_es_MONEDONOW_IL_y =divid_es_MONEDONOW_IL.sort_values(by=['band'],ascending = False)
divid_es_MONEDONOW_IL_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04
band,,,
f_0.73-1,0.046875,0.040000,0.028571
e_0.71-0.73,0.045455,0.027778,0.000000
d_0.68-0.71,0.040816,0.023810,0.062500
c_0.65-0.68,0.051282,0.029703,0.051282
b_0-0.65,0.120000,0.085470,0.170213


# spain_excel_sheet

In [24]:
wb = load_workbook('Model_score_outPut.xlsx')

# Get sheet names
print(wb.sheetnames)

['Model_Score_Distribution_RU', 'Model_Score_Distribution_ES', 'Model_Score_Distribution_PL']


In [25]:
path = r"C:\Users\abdalla.khiralla\Model_score_outPut.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

Lead_MONEDONOW_ES_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=6,startcol=1,header=True)
OFFER_MONEDONOW_ES_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=6,startcol=10,header=True)
PAYOUT_MONEDONOW_ES_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=6,startcol=19,header=True)
Lead_ES_k24_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=52,startcol=1,header=True)
offer_ES_K24_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=52,startcol=10,header=True)
PAYOUT_ES_K24_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=52,startcol=19,header=True)
Lead_ES_K24_Rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=99,startcol=1,header=True)
offer_ES_K24_rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=99,startcol=10,header=True)
PAYOUT_ES_K24_rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=99,startcol=19,header=True)
divid_es_ML_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=78,startcol=19,header=True)
divid_es_ml_rec_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=125,startcol=19,header=True)
divid_es_MONEDONOW_IL_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_ES',startrow=31,startcol=19,header=True)

writer.save()
writer.close()

# 2.Russia

## 2.1 Russia-New

In [26]:
ZAIMO = row_data.loc[row_data.CLIENT =='ZAIMO']
bins= [0,0.56,0.62,0.68,0.78,1]
labels = ['b_0-0.56','c_0.56-0.62','d_0.62-0.68','e_0.68-0.78','f_0.78-1']
ZAIMO['band'] = pd.cut(ZAIMO['SCORE'], bins=bins, labels=labels, right=False)
ZAIMO.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
6,2020-02,ZAIMO,RECURRING,2020-02-12 10:01:28.132000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 10:20:21.598000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1033631R11,NaN
7,2020-02,ZAIMO,RECURRING,2020-02-13 00:56:27.025000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-13 01:01:19.929000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1036195R11,NaN
14,2020-02,ZAIMO,NEW,2020-02-12 08:37:32.882000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 08:57:03.294000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1033430R11,NaN
21,2020-02,ZAIMO,NEW,2020-02-12 15:20:38.616000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 15:30:26.511000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1034775R11,NaN
24,2020-02,ZAIMO,NEW,2020-02-12 11:14:11.252000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 11:20:12.865000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1033862R11,NaN


## LEADD

In [27]:
Lead_ZAIMO = ZAIMO.loc[(ZAIMO['LEADD'] == 1) & (ZAIMO['NEW_RECURRING'] =='NEW') & (ZAIMO['SCORING_MODEL'] == 'RU-K24-MED-NEW-PRED-CB-PA-ST-m21e8a3-1')]
Lead_ZAIMO_outPut= pd.pivot_table(Lead_ZAIMO, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
Lead_ZAIMO_outPut_x = Lead_ZAIMO_outPut.copy()
Lead_ZAIMO_outPut_x.columns = [''.join(col).strip() for col in Lead_ZAIMO_outPut.columns.values]
Lead_ZAIMO_outPut_y= Lead_ZAIMO_outPut_x.sort_values(by=['band'],ascending = False)
Lead_ZAIMO_outPut_y

,LEADD2020-02,LEADD2020-03,LEADD2020-04,LEADD2020-05,LEADD2020-06,LEADD2020-07
band,,,,,,
f_0.78-1,569,285,406,353,256,108
e_0.68-0.78,613,419,602,485,334,164
d_0.62-0.68,543,359,496,451,299,123
c_0.56-0.62,624,479,655,545,416,183
b_0-0.56,16255,19266,22133,14770,11598,5021


## OFFER

In [28]:
OFFER_ZAIMO = ZAIMO.loc[(ZAIMO['OFFER'] == 1) & (ZAIMO['NEW_RECURRING'] =='NEW') & (ZAIMO['SCORING_MODEL'] == 'RU-K24-MED-NEW-PRED-CB-PA-ST-m21e8a3-1')]
OFFER_ZAIMO_outPut= pd.pivot_table(OFFER_ZAIMO, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
OFFER_ZAIMO_outPut_x = OFFER_ZAIMO_outPut.copy()
OFFER_ZAIMO_outPut_x.columns = [''.join(col).strip() for col in OFFER_ZAIMO_outPut.columns.values]
OFFER_ZAIMO_outPut_y= OFFER_ZAIMO_outPut_x.sort_values(by=['band'],ascending = False)

## PAYOUT

In [29]:
PAYOUT_ZAIMO = ZAIMO.loc[(ZAIMO['PAYOUT'] == 1) & (ZAIMO['NEW_RECURRING'] =='NEW') &(ZAIMO['SCORING_MODEL'] == 'RU-K24-MED-NEW-PRED-CB-PA-ST-m21e8a3-1')]
PAYOUT_ZAIMO_outPut= pd.pivot_table(PAYOUT_ZAIMO, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PAYOUT_ZAIMO_outPut_x = PAYOUT_ZAIMO_outPut.copy()
PAYOUT_ZAIMO_outPut_x.columns = [''.join(col).strip() for col in PAYOUT_ZAIMO_outPut.columns.values]
PAYOUT_ZAIMO_outPut_y= PAYOUT_ZAIMO_outPut_x.sort_values(by=['band'],ascending = False)
PAYOUT_ZAIMO_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.78-1,395,220,336,231,211,74
e_0.68-0.78,402,323,476,326,259,126
d_0.62-0.68,349,269,397,298,246,94
c_0.56-0.62,370,377,518,364,331,144
b_0-0.56,452,135,212,146,122,70


# DPD 30+

In [30]:
Dpd_Ru_Ml =  PAYOUT_ZAIMO.loc[(PAYOUT_ZAIMO['MONTHS_SINCE_PAYOUT'] == 2) & (PAYOUT_ZAIMO['DPD_EOM'] >= 30)]
Dpd_Ru_Ml_outPut= pd.pivot_table(Dpd_Ru_Ml, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_Ru_Ml_outPut.columns = [''.join(col).strip() for col in Dpd_Ru_Ml_outPut.columns.values]
Dpd_Ru_Ml_outPut_x= Dpd_Ru_Ml_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_Ru_Ml_outPut= Dpd_Ru_Ml_outPut_x/PAYOUT_ZAIMO_outPut_y
divid_Dpd_Ru_Ml_outPut_y =divid_Dpd_Ru_Ml_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_Ru_Ml_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.78-1,0.108861,0.050000,0.053571,0.051948,0.028436,NaN
e_0.68-0.78,0.169154,0.114551,0.138655,0.110429,0.077220,NaN
d_0.62-0.68,0.240688,0.126394,0.171285,0.164430,0.073171,NaN
c_0.56-0.62,0.254054,0.180371,0.227799,0.178571,0.084592,NaN
b_0-0.56,0.396018,0.237037,0.245283,0.226027,0.131148,NaN


# 2.2 Russia-Rec

In [31]:
ZAIMO_rec = row_data.loc[row_data.CLIENT =='ZAIMO']
bins= [0,0.57,0.70,0.79,0.88,1]
labels = ['b_0-0.57','c_0.57-0.70','d_0.70-0.79','e_0.79-0.88','f_0.88-1']
ZAIMO_rec['band'] = pd.cut(ZAIMO_rec['SCORE'], bins=bins, labels=labels, right=False)
ZAIMO_rec.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
6,2020-02,ZAIMO,RECURRING,2020-02-12 10:01:28.132000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 10:20:21.598000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1033631R11,NaN
7,2020-02,ZAIMO,RECURRING,2020-02-13 00:56:27.025000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-13 01:01:19.929000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1036195R11,NaN
14,2020-02,ZAIMO,NEW,2020-02-12 08:37:32.882000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 08:57:03.294000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1033430R11,NaN
21,2020-02,ZAIMO,NEW,2020-02-12 15:20:38.616000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 15:30:26.511000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1034775R11,NaN
24,2020-02,ZAIMO,NEW,2020-02-12 11:14:11.252000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 11:20:12.865000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLR1033862R11,NaN


## LEADD-Rec

In [32]:
Lead_ZAIMO_Rec = ZAIMO_rec.loc[(ZAIMO_rec['LEADD'] == 1) & (ZAIMO_rec['NEW_RECURRING'] =='RECURRING') & (ZAIMO_rec['SCORING_MODEL'] == 'RU-K24-MED-REC_PR-PRED-CB-PA-ST-me76543-1')]
Lead_ZAIMO_Rec_outPut= pd.pivot_table(Lead_ZAIMO_Rec, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
Lead_ZAIMO_Rec_outPut_x = Lead_ZAIMO_Rec_outPut.copy()
Lead_ZAIMO_Rec_outPut_x.columns = [''.join(col).strip() for col in Lead_ZAIMO_Rec_outPut.columns.values]
Lead_ZAIMO_Rec_outPut_y= Lead_ZAIMO_Rec_outPut_x.sort_values(by=['band'],ascending = False)
Lead_ZAIMO_Rec_outPut_y

,LEADD2020-02,LEADD2020-03,LEADD2020-04,LEADD2020-05,LEADD2020-06,LEADD2020-07
band,,,,,,
f_0.88-1,1668,1945,1553,1743,1668,1826
e_0.79-0.88,2034,2303,1851,1996,1940,1925
d_0.70-0.79,1669,1885,1425,1539,1473,1594
c_0.57-0.70,1951,2039,1564,1626,1558,1600
b_0-0.57,2246,2360,1918,1760,1565,1654


## OFFER-Rec

In [33]:
OFFER_ZAIMO_Rec = ZAIMO_rec.loc[(ZAIMO_rec['OFFER'] == 1) & (ZAIMO_rec['NEW_RECURRING'] =='RECURRING') & (ZAIMO_rec['SCORING_MODEL'] == 'RU-K24-MED-REC_PR-PRED-CB-PA-ST-me76543-1')]
OFFER_ZAIMO_Rec_outPut= pd.pivot_table(OFFER_ZAIMO_Rec, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
OFFER_ZAIMO_Rec_outPut_x = OFFER_ZAIMO_Rec_outPut.copy()
OFFER_ZAIMO_Rec_outPut_x.columns = [''.join(col).strip() for col in OFFER_ZAIMO_Rec_outPut.columns.values]
OFFER_ZAIMO_Rec_outPut_y= OFFER_ZAIMO_Rec_outPut_x.sort_values(by=['band'],ascending = False)
OFFER_ZAIMO_Rec_outPut_y

,OFFER2020-02,OFFER2020-03,OFFER2020-04,OFFER2020-05,OFFER2020-06,OFFER2020-07
band,,,,,,
f_0.88-1,1635,1915,1528,1719,1650,1775
e_0.79-0.88,2006,2272,1822,1980,1916,1892
d_0.70-0.79,1648,1863,1419,1526,1444,1580
c_0.57-0.70,1932,2022,1545,1620,1546,1580
b_0-0.57,1886,1831,1443,1351,1209,1177


## PAYOUT-Rec

In [34]:
PAYOUT_ZAIMO_Rec = ZAIMO_rec.loc[(ZAIMO_rec['PAYOUT'] == 1) & (ZAIMO_rec['NEW_RECURRING'] =='RECURRING') & (ZAIMO_rec['SCORING_MODEL'] == 'RU-K24-MED-REC_PR-PRED-CB-PA-ST-me76543-1')]
PAYOUT_ZAIMO_Rec_outPut= pd.pivot_table(PAYOUT_ZAIMO_Rec, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PAYOUT_ZAIMO_Rec_outPut_x = PAYOUT_ZAIMO_Rec_outPut.copy()
PAYOUT_ZAIMO_Rec_outPut_x.columns = [''.join(col).strip() for col in PAYOUT_ZAIMO_Rec_outPut.columns.values]
PAYOUT_ZAIMO_Rec_outPut_y= PAYOUT_ZAIMO_Rec_outPut_x.sort_values(by=['band'],ascending = False)
PAYOUT_ZAIMO_Rec_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.88-1,1604,1886,1493,1570,1602,1732
e_0.79-0.88,1961,2217,1772,1747,1816,1793
d_0.70-0.79,1598,1811,1363,1334,1374,1485
c_0.57-0.70,1859,1948,1450,1468,1467,1484
b_0-0.57,1805,1764,1366,1251,1129,1113


# DPD30+

In [35]:
Dpd_Ru_Ml_rec =  PAYOUT_ZAIMO_Rec.loc[(PAYOUT_ZAIMO_Rec['MONTHS_SINCE_PAYOUT'] == 2) & (PAYOUT_ZAIMO_Rec['DPD_EOM'] >= 30)]
Dpd_Ru_Ml_rec_outPut= pd.pivot_table(Dpd_Ru_Ml_rec, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_Ru_Ml_rec_outPut.columns = [''.join(col).strip() for col in Dpd_Ru_Ml_rec_outPut.columns.values]
Dpd_Ru_Ml_rec_outPut_x= Dpd_Ru_Ml_rec_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_Ru_Ml_rec_outPut= Dpd_Ru_Ml_rec_outPut_x/PAYOUT_ZAIMO_Rec_outPut_y
divid_Dpd_Ru_Ml_rec_outPut_y =divid_Dpd_Ru_Ml_rec_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_Ru_Ml_rec_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.88-1,0.028055,0.044008,0.034829,0.029299,0.012484,NaN
e_0.79-0.88,0.083121,0.100586,0.077314,0.052089,0.030286,NaN
d_0.70-0.79,0.150814,0.166207,0.151137,0.098201,0.053130,NaN
c_0.57-0.70,0.202797,0.233060,0.189655,0.141008,0.060668,NaN
b_0-0.57,0.322438,0.344671,0.278917,0.227018,0.108060,NaN


## Russian_excel_sheet

In [36]:
path = r"C:\Users\abdalla.khiralla\Model_score_outPut.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

Lead_ZAIMO_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_RU',startrow=6,startcol=1,header=True)
OFFER_ZAIMO_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_RU',startrow=6,startcol=10,header=True)
PAYOUT_ZAIMO_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_RU',startrow=6,startcol=19,header=True)
Lead_ZAIMO_Rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_RU',startrow=49,startcol=1,header=True)
OFFER_ZAIMO_Rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_RU',startrow=49,startcol=10,header=True)
PAYOUT_ZAIMO_Rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_RU',startrow=49,startcol=19,header=True)
divid_Dpd_Ru_Ml_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_RU',startrow=32,startcol=19,header=True)
divid_Dpd_Ru_Ml_rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_RU',startrow=75,startcol=19,header=True)

writer.save()
writer.close()

# 3. a Poland_IL

## NEW_IL

In [37]:
PL_IL = row_data.loc[row_data.CLIENT =='MONEDONOW_PL']
bins= [0,0.673,0.714,0.756,0.814,1]
labels = ['b_0-0.673','c_0.673-0.714','d_0.714-0.756','e_0.756-0.814','f_0.814-1']
PL_IL['band'] = pd.cut(PL_IL['SCORE'], bins=bins, labels=labels, right=False)
PL_IL.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)
PL_IL.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
0,2020-02,MONEDONOW_PL,NEW,2020-02-12 17:39:09.682000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-12 17:50:51.018000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DLP2186722R11,NaN
2727,2020-02,MONEDONOW_PL,NEW,2020-02-04 14:21:48.497000,PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1,0.22906,2020-02-04 14:32:03.818000,1.0,NaN,NaN,2020-02-04 14:32:25.950000,1.0,NaN,NaN,NaN,NaN,NEW,DLP2143336R11,NaN,NaN,DLP2143336R11,b_0-0.673
2728,2020-02,MONEDONOW_PL,NEW,2020-02-04 17:13:10.374000,PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1,0.51103,2020-02-04 17:17:49.283000,1.0,NaN,NaN,2020-02-04 17:18:00.661000,1.0,NaN,NaN,NaN,NaN,NEW,DLP2144530R11,NaN,NaN,DLP2144530R11,b_0-0.673
2729,2020-02,MONEDONOW_PL,NEW,2020-02-04 12:46:34.649000,PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1,0.27772,2020-02-04 12:51:22.447000,1.0,NaN,NaN,2020-02-04 12:51:38.277000,1.0,NaN,NaN,NaN,NaN,NEW,DLP2142736R11,NaN,NaN,DLP2142736R11,b_0-0.673
2730,2020-02,MONEDONOW_PL,NEW,2020-02-04 20:01:03.690000,PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1,0.20680,2020-02-04 20:14:22.527000,1.0,NaN,NaN,2020-02-04 20:14:33.753000,1.0,NaN,NaN,NaN,NaN,NEW,DLP2145452R11,NaN,NaN,DLP2145452R11,b_0-0.673


## LEADD

In [38]:
PL_IL_new= PL_IL[PL_IL.NEW_RECURRING != 'RECURRING']

In [39]:
Lead_PL_IL = PL_IL_new.loc[(PL_IL_new['LEADD'] == 1) & ( PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0')|(PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423')|(PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
Lead_PL_IL_outPut= pd.pivot_table(Lead_PL_IL, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
Lead_PL_IL_outPut_x = Lead_PL_IL_outPut.copy()
Lead_PL_IL_outPut_x.columns = [''.join(col).strip() for col in Lead_PL_IL_outPut.columns.values]
Lead_PL_IL_outPut_y= Lead_PL_IL_outPut_x.sort_values(by=['band'],ascending = False)

## OFFER

In [40]:
offer_PL_IL = PL_IL_new.loc[(PL_IL_new['OFFER'] == 1) & ( PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0')|(PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423')|(PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
offer_PL_IL_outPut= pd.pivot_table(offer_PL_IL, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
offer_PL_IL_outPut_x = offer_PL_IL_outPut.copy()
offer_PL_IL_outPut_x.columns = [''.join(col).strip() for col in offer_PL_IL_outPut.columns.values]
offer_PL_IL_outPut_y= offer_PL_IL_outPut_x.sort_values(by=['band'],ascending = False)

## PAYOUT

In [41]:
payout_PL_IL = PL_IL_new.loc[(PL_IL_new['PAYOUT'] == 1) & ( PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0') | (PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423') | (PL_IL_new['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
payout_PL_IL_outPut= pd.pivot_table(payout_PL_IL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
payout_PL_IL_outPut_x = payout_PL_IL_outPut.copy()
payout_PL_IL_outPut_x.columns = [''.join(col).strip() for col in payout_PL_IL_outPut.columns.values]
payout_PL_IL_outPut_y= payout_PL_IL_outPut_x.sort_values(by=['band'],ascending = False)

# DPD 30+ PL IL NEW

In [42]:
Dpd_IL_new =  payout_PL_IL.loc[(payout_PL_IL['MONTHS_SINCE_PAYOUT'] == 3) & (payout_PL_IL['DPD_EOM'] >= 30)]
Dpd_IL_new_outPut= pd.pivot_table(Dpd_IL_new, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_IL_new_outPut.columns = [''.join(col).strip() for col in Dpd_IL_new_outPut.columns.values]
Dpd_IL_new_outPut_x= Dpd_IL_new_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_IL_new_outPut= Dpd_IL_new_outPut_x/payout_PL_IL_outPut_y
divid_Dpd_IL_new_outPut_y =divid_Dpd_IL_new_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_IL_new_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.814-1,0.008772,0.016129,0.011299,0.000000,NaN,NaN
e_0.756-0.814,0.012500,0.033520,0.008969,0.003984,NaN,NaN
d_0.714-0.756,0.021622,0.030000,0.027650,0.008097,NaN,NaN
c_0.673-0.714,0.037915,0.017167,0.011407,0.019108,NaN,NaN
b_0-0.673,0.095455,0.079570,0.000000,0.000000,NaN,NaN


# Rec_IL

In [43]:
PL_IL_rec = row_data.loc[row_data.CLIENT =='MONEDONOW_PL']
bins= [0,0.468,0.567,0.650,0.740,1]
labels = ['b_0-0.468','c_0.468-0.567','d_0.567-0.650','e_0.650-0.740','f_0.740-1']
PL_IL_rec['band'] = pd.cut(PL_IL_rec['SCORE'], bins=bins, labels=labels, right=False)
PL_IL_rec.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)


In [44]:
PL_IL_Rec_x= PL_IL_rec[(PL_IL_rec.NEW_RECURRING != 'NEW') & (PL_IL_rec['CUSTOMER_TYPE_DETAILED'] == "REGULAR_PRODUCT_REPEAT")]

In [45]:
PL_IL_Rec_x.NEW_RECURRING.value_counts()

RECURRING    2913
Name: NEW_RECURRING, dtype: int64

## LEADD-Rec

In [46]:
PL_IL_Rec_y = PL_IL_Rec_x.loc[(PL_IL_Rec_x['LEADD'] == 1) & ( PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0')|(PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423')|(PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
PL_IL_Rec_outPut= pd.pivot_table(PL_IL_Rec_y, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_IL_Rec_outPut_x = PL_IL_Rec_outPut.copy()
PL_IL_Rec_outPut_x.columns = [''.join(col).strip() for col in PL_IL_Rec_outPut.columns.values]
PL_IL_Rec_outPut_y= PL_IL_Rec_outPut_x.sort_values(by=['band'],ascending = False)
PL_IL_Rec_outPut_y

,LEADD2020-02,LEADD2020-03,LEADD2020-04,LEADD2020-05,LEADD2020-06
band,,,,,
f_0.740-1,219,123,117,81,2
e_0.650-0.740,203,172,88,81,1
d_0.567-0.650,184,140,94,70,0
c_0.468-0.567,122,132,102,67,0
b_0-0.468,228,314,236,132,0


## OFFER-Rec

In [47]:
offer_PL_IL_Rec = PL_IL_Rec_x.loc[(PL_IL_Rec_x['OFFER'] == 1) & ( PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0')|(PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423')|(PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
offer_PL_IL_Rec_outPut= pd.pivot_table(offer_PL_IL_Rec, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
offer_PL_IL_Rec_outPut_x = offer_PL_IL_Rec_outPut.copy()
offer_PL_IL_Rec_outPut_x.columns = [''.join(col).strip() for col in offer_PL_IL_Rec_outPut.columns.values]
offer_PL_IL_Rec_outPut_y= offer_PL_IL_Rec_outPut_x.sort_values(by=['band'],ascending = False)
offer_PL_IL_Rec_outPut_y

,OFFER2020-02,OFFER2020-03,OFFER2020-04,OFFER2020-05,OFFER2020-06
band,,,,,
f_0.740-1,160,92,83,67,0
e_0.650-0.740,156,123,62,63,0
d_0.567-0.650,149,104,67,61,0
c_0.468-0.567,96,99,20,9,0
b_0-0.468,37,109,0,0,0


# payout-Rec

In [48]:
payout_PL_IL_Rec = PL_IL_Rec_x.loc[(PL_IL_Rec_x['PAYOUT'] == 1) & ( PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0') | (PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423') | (PL_IL_Rec_x['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
payout_PL_IL_Rec_outPut= pd.pivot_table(payout_PL_IL_Rec, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
payout_PL_IL_Rec_x = payout_PL_IL_Rec_outPut.copy()
payout_PL_IL_Rec_x.columns = [''.join(col).strip() for col in payout_PL_IL_Rec_outPut.columns.values]
payout_PL_IL_Rec_y= payout_PL_IL_Rec_x.sort_values(by=['band'],ascending = False)
payout_PL_IL_Rec_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06
band,,,,,
f_0.740-1,126,74,75,60,0
e_0.650-0.740,115,75,52,50,0
d_0.567-0.650,91,69,54,45,0
c_0.468-0.567,71,63,14,9,0
b_0-0.468,27,89,0,0,0


# DPD 30+

In [49]:
Dpd_IL_rec =  payout_PL_IL_Rec.loc[(payout_PL_IL_Rec['MONTHS_SINCE_PAYOUT'] == 3) & (payout_PL_IL_Rec['DPD_EOM'] >= 30)]
Dpd_IL_rec_outPut= pd.pivot_table(Dpd_IL_rec, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_IL_rec_outPut.columns = [''.join(col).strip() for col in Dpd_IL_rec_outPut.columns.values]
Dpd_IL_rec_outPut_x= Dpd_IL_rec_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_IL_rec_outPut= Dpd_IL_rec_outPut_x/payout_PL_IL_Rec_y
divid_Dpd_IL_rec_outPut_y =divid_Dpd_IL_rec_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_IL_rec_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06
band,,,,,
f_0.740-1,0.031746,0.000000,0.026667,0.000000,NaN
e_0.650-0.740,0.017391,0.026667,0.019231,0.000000,NaN
d_0.567-0.650,0.054945,0.000000,0.000000,0.044444,NaN
c_0.468-0.567,0.084507,0.063492,0.071429,0.000000,NaN
b_0-0.468,0.111111,0.078652,NaN,NaN,NaN


# Rec_IL_CROSS_SALE_LAAS

In [50]:
PL_IL_rec = row_data.loc[row_data.CLIENT =='MONEDONOW_PL']
bins= [0,0.468,0.567,0.650,0.740,1]
labels = ['b_0-0.468','c_0.468-0.567','d_0.567-0.650','e_0.650-0.740','f_0.740-1']
PL_IL_rec['band'] = pd.cut(PL_IL_rec['SCORE'], bins=bins, labels=labels, right=False)
PL_IL_rec.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)

In [51]:
PL_IL_Rec_cross = PL_IL_rec[(PL_IL_rec.NEW_RECURRING != 'NEW') & (PL_IL_rec['CUSTOMER_TYPE_DETAILED'] == "CROSS_SALE_LAAS")]

# Leadd

In [52]:
PL_IL_Rec_z = PL_IL_Rec_cross.loc[(PL_IL_Rec_cross['LEADD'] == 1) & ( PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0')|(PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423')|(PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
PL_IL_Rec_cross_outPut= pd.pivot_table(PL_IL_Rec_z, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_IL_Rec_cross_outPut_x = PL_IL_Rec_cross_outPut.copy()
PL_IL_Rec_cross_outPut_x.columns = [''.join(col).strip() for col in PL_IL_Rec_cross_outPut.columns.values]
PL_IL_Rec_cross_outPut= PL_IL_Rec_cross_outPut_x.sort_values(by=['band'],ascending = False)
PL_IL_Rec_cross_outPut

,LEADD2020-02,LEADD2020-03,LEADD2020-04,LEADD2020-05,LEADD2020-06,LEADD2020-07
band,,,,,,
f_0.740-1,90,52,252,337,300,316
e_0.650-0.740,76,70,203,245,214,228
d_0.567-0.650,107,51,179,242,215,209
c_0.468-0.567,92,96,183,216,215,214
b_0-0.468,152,180,392,409,345,367


# Offer

In [53]:
offer_PL_IL_Rec_cross = PL_IL_Rec_cross.loc[(PL_IL_Rec_cross['OFFER'] == 1) & ( PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0')|(PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423')|(PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
offer_PL_IL_Rec_cross_outPut= pd.pivot_table(offer_PL_IL_Rec_cross, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
offer_PL_IL_Rec_outPut_cross_x = offer_PL_IL_Rec_cross_outPut.copy()
offer_PL_IL_Rec_outPut_cross_x.columns = [''.join(col).strip() for col in offer_PL_IL_Rec_outPut_cross_x.columns.values]
offer_PL_IL_Rec_outPut_cross_y= offer_PL_IL_Rec_outPut_cross_x.sort_values(by=['band'],ascending = False)
offer_PL_IL_Rec_outPut_cross_y

,OFFER2020-02,OFFER2020-03,OFFER2020-04,OFFER2020-05,OFFER2020-06,OFFER2020-07
band,,,,,,
f_0.740-1,63,39,126,182,154,158
e_0.650-0.740,55,43,109,125,117,118
d_0.567-0.650,73,38,115,143,122,100
c_0.468-0.567,74,74,85,111,117,111
b_0-0.468,32,59,34,23,0,0


# PayOUt

In [54]:
payout_PL_IL_Rec_cross = PL_IL_Rec_cross.loc[(PL_IL_Rec_cross['PAYOUT'] == 1) & ( PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1.0') | (PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423') | (PL_IL_Rec_cross['SCORING_MODEL'] == 'PL-MNOW-HIGH-NEW-PRED-KK_CB-MS-md89423-1')]
payout_PL_IL_Rec_cross_outPut= pd.pivot_table(payout_PL_IL_Rec_cross, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
payout_PL_IL_Rec_cross_outPut_x = payout_PL_IL_Rec_cross_outPut.copy()
payout_PL_IL_Rec_cross_outPut_x.columns = [''.join(col).strip() for col in payout_PL_IL_Rec_cross_outPut.columns.values]
payout_PL_IL_Rec_cross_outPut_y= payout_PL_IL_Rec_cross_outPut_x.sort_values(by=['band'],ascending = False)
payout_PL_IL_Rec_cross_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.740-1,48,26,106,173,142,143
e_0.650-0.740,34,35,87,116,106,100
d_0.567-0.650,48,31,93,111,96,82
c_0.468-0.567,51,52,60,81,85,90
b_0-0.468,23,44,31,21,0,0


# DPD 30+

In [55]:
Dpd_IL_rec_cross =  payout_PL_IL_Rec_cross.loc[(payout_PL_IL_Rec_cross['MONTHS_SINCE_PAYOUT'] == 3) & (payout_PL_IL_Rec_cross['DPD_EOM'] >= 30)]
Dpd_IL_rec_cross_outPut= pd.pivot_table(Dpd_IL_rec_cross, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_IL_rec_cross_outPut.columns = [''.join(col).strip() for col in Dpd_IL_rec_cross_outPut.columns.values]
Dpd_IL_rec_cross_outPut_x= Dpd_IL_rec_cross_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_IL_rec_cross_outPut= Dpd_IL_rec_cross_outPut_x.div(payout_PL_IL_Rec_cross_outPut_y)
divid_Dpd_IL_rec_cross_outPut_y =divid_Dpd_IL_rec_cross_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_IL_rec_cross_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.740-1,0.020833,0.038462,0.000000,0.000000,NaN,NaN
e_0.650-0.740,0.000000,0.028571,0.000000,0.008621,NaN,NaN
d_0.567-0.650,0.000000,0.064516,0.010753,0.018018,NaN,NaN
c_0.468-0.567,0.039216,0.019231,0.016667,0.024691,NaN,NaN
b_0-0.468,0.043478,0.022727,0.000000,0.095238,NaN,NaN


# 3.b Poland BNpl

In [56]:
PL_BNPL = row_data.loc[row_data.CLIENT =='MONEDO_BNPL_PL']
bins= [0,0.88,0.92,0.95,0.97,1]
labels = ['b_0-0.88','c_0.88-0.92','d_0.92-0.95','e_0.95-0.97','f_0.97-1']
PL_BNPL['band'] = pd.cut(PL_BNPL['SCORE'], bins=bins, labels=labels, right=False)
#PL_BNPL.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)
PL_BNPL.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
106,2020-02,MONEDO_BNPL_PL,RECURRING,2020-02-13 20:55:40.535000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-13 20:56:07.110000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLP1674805R11,NaN
139,2020-02,MONEDO_BNPL_PL,NEW,2020-02-13 06:46:23.575000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLP1673157R11,NaN
149,2020-02,MONEDO_BNPL_PL,NEW,2020-02-13 22:22:44.036000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-13 22:23:13.393000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLP1675005R11,NaN
161,2020-02,MONEDO_BNPL_PL,NEW,2020-02-13 20:02:22.264000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-13 20:04:34.339000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLP1674093R11,NaN
167,2020-02,MONEDO_BNPL_PL,NEW,2020-02-13 18:57:30.589000,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-13 18:58:03.348000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLP1674519R11,NaN


## LEADD-BNPL

In [57]:
PL_BNPL_leads = PL_BNPL.loc[(PL_BNPL['LEADD'] == 1) & ( PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')|(PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')]
PL_BNPL_leads_outPut= pd.pivot_table(PL_BNPL_leads, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_BNPL_leads_outPut_x = PL_BNPL_leads_outPut.copy()
PL_BNPL_leads_outPut_x.columns = [''.join(col).strip() for col in PL_BNPL_leads_outPut.columns.values]
PL_BNPL_leads_outPut_y= PL_BNPL_leads_outPut_x.sort_values(by=['band'],ascending = False)

## OFFER-BNPL

In [58]:
PL_OFFER_BNPL = PL_BNPL.loc[(PL_BNPL['OFFER'] == 1) & ( PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')|(PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')]
PL_OFFER_BNPL_outPut= pd.pivot_table(PL_BNPL_leads, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_OFFER_BNPL_outPut_x = PL_OFFER_BNPL_outPut.copy()
PL_OFFER_BNPL_outPut_x.columns = [''.join(col).strip() for col in PL_OFFER_BNPL_outPut.columns.values]
PL_OFFER_BNPL_outPut_y= PL_OFFER_BNPL_outPut_x.sort_values(by=['band'],ascending = False)

## payout-BNPL

In [59]:
PL_BNPL_PAYOUT = PL_BNPL.loc[(PL_BNPL['PAYOUT'] == 1) & ( PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')|(PL_BNPL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')]
PL_BNPL_PAYOUT_outPut= pd.pivot_table(PL_BNPL_leads, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_BNPL_PAYOUT_outPut_x = PL_BNPL_PAYOUT_outPut.copy()
PL_BNPL_PAYOUT_outPut_x.columns = [''.join(col).strip() for col in PL_BNPL_PAYOUT_outPut.columns.values]
PL_BNPL_PAYOUT_outPut_y= PL_BNPL_PAYOUT_outPut_x.sort_values(by=['band'],ascending = False)

# DPD 30+ BNPL

In [60]:
Dpd_BNPL =  PL_BNPL_PAYOUT.loc[(PL_BNPL_PAYOUT['MONTHS_SINCE_PAYOUT'] == 3) & (PL_BNPL_PAYOUT['DPD_EOM'] >= 30)]
Dpd_BNPL_outPut= pd.pivot_table(Dpd_BNPL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_BNPL_outPut.columns = [''.join(col).strip() for col in Dpd_BNPL_outPut.columns.values]
Dpd_BNPL_outPut_x= Dpd_BNPL_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_BNPL_outPut= Dpd_BNPL_outPut_x/PL_BNPL_PAYOUT_outPut_y
divid_Dpd_BNPL_outPut_y = divid_Dpd_BNPL_outPut.sort_values(by=['band'],ascending = False)
divid_Dpd_BNPL_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.97-1,0.000000,0.002165,0.005102,0.000000,NaN,NaN
e_0.95-0.97,0.003559,0.000000,0.013298,0.014970,NaN,NaN
d_0.92-0.95,0.000000,0.015152,0.014327,0.005540,NaN,NaN
c_0.88-0.92,0.014085,0.018315,0.014706,0.003846,NaN,NaN
b_0-0.88,0.022989,0.027919,0.032663,0.048276,NaN,NaN


# 4. MF - 0% POS

## 4.a HIGH RISK MODEL 

In [61]:
PL_HIGH_RISK = row_data.loc[row_data.CLIENT =='MONEDO_FINANCE_PL']
PL_HIGH_RISK_MODEL= PL_HIGH_RISK[(PL_HIGH_RISK.CUSTOMER_TYPE_DETAILED == '0%_FINANCING - BANK_REJECTED_POS') | (PL_HIGH_RISK.CUSTOMER_TYPE_DETAILED == '0%_FINANCING_POS')]
bins= [0,0.977,0.984,0.988,0.992,1]
labels = ['b_0-0.977','c_0.977-0.984','d_0.984-0.988','e_0.988-0.992','f_0.992-1']
PL_HIGH_RISK_MODEL['band'] = pd.cut(PL_HIGH_RISK_MODEL['SCORE'], bins=bins, labels=labels, right=False)
PL_HIGH_RISK_MODEL.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)
PL_HIGH_RISK_MODEL.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
2726,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-23 20:28:27.109000,PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1,0.89192,2020-02-23 20:31:16.074000,1.0,NaN,NaN,2020-02-23 20:31:18.596000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1693895R11,NaN,NaN,ZLP1693895R11,b_0-0.977
2771,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-10 21:49:09.305000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.95565,2020-02-10 21:56:06.150000,1.0,NaN,NaN,2020-02-10 21:56:13.146000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1668381R11,NaN,NaN,ZLP1668381R11,b_0-0.977
2811,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 17:16:32.818000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.93237,2020-02-06 09:53:57.418000,1.0,NaN,NaN,2020-02-06 09:54:19.543000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1658199R11,NaN,NaN,ZLP1658199R11,b_0-0.977
2814,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 15:10:14.386000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.95848,2020-02-05 15:35:45.284000,1.0,NaN,NaN,2020-02-05 15:35:54.704000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1657907R11,NaN,NaN,ZLP1657907R11,b_0-0.977
2826,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-08 14:42:12.625000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.84603,2020-02-08 14:47:14.824000,1.0,NaN,NaN,2020-02-08 14:47:24.564000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING_POS,ZLP1663375R11,NaN,NaN,ZLP1663375R11,b_0-0.977


### LEADD-HIGH RISK MODEL-0% POS

In [62]:
PL_HIGH_RISK_MODEL_leadd = PL_HIGH_RISK_MODEL.loc[(PL_HIGH_RISK_MODEL['LEADD'] == 1) & (( PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1.0')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3'))]
PL_HIGH_RISK_MODEL_leadd_outPut= pd.pivot_table(PL_HIGH_RISK_MODEL_leadd, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_HIGH_RISK_MODEL_leadd_outPut_x = PL_HIGH_RISK_MODEL_leadd_outPut.copy()
PL_HIGH_RISK_MODEL_leadd_outPut_x.columns = [''.join(col).strip() for col in PL_HIGH_RISK_MODEL_leadd_outPut.columns.values]
PL_HIGH_RISK_MODEL_leadd_outPut_y= PL_HIGH_RISK_MODEL_leadd_outPut_x.sort_values(by=['band'],ascending = False)

### OFFER-HIGH RISK MODEL-0% POS

In [63]:
PL_HIGH_RISK_MODEL_OFFER = PL_HIGH_RISK_MODEL.loc[(PL_HIGH_RISK_MODEL['OFFER'] == 1) & (( PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1.0')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3'))]
PL_HIGH_RISK_MODEL_OFFER_outPut= pd.pivot_table(PL_HIGH_RISK_MODEL_leadd, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_HIGH_RISK_MODEL_OFFER_outPut_x = PL_HIGH_RISK_MODEL_OFFER_outPut.copy()
PL_HIGH_RISK_MODEL_OFFER_outPut_x.columns = [''.join(col).strip() for col in PL_HIGH_RISK_MODEL_OFFER_outPut.columns.values]
PL_HIGH_RISK_MODEL_OFFER_outPut_y= PL_HIGH_RISK_MODEL_OFFER_outPut_x.sort_values(by=['band'],ascending = False)

### payout-HIGH RISK MODEL-0% POS

In [64]:
PL_HIGH_RISK_MODEL_payout = PL_HIGH_RISK_MODEL.loc[(PL_HIGH_RISK_MODEL['PAYOUT'] == 1) & (( PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1.0')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3'))]
PL_HIGH_RISK_MODEL_payout_outPut= pd.pivot_table(PL_HIGH_RISK_MODEL_leadd, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_HIGH_RISK_MODEL_payout_outPut_x = PL_HIGH_RISK_MODEL_payout_outPut.copy()
PL_HIGH_RISK_MODEL_payout_outPut_x.columns = [''.join(col).strip() for col in PL_HIGH_RISK_MODEL_payout_outPut.columns.values]
PL_HIGH_RISK_MODEL_payout_outPut_y= PL_HIGH_RISK_MODEL_payout_outPut_x.sort_values(by=['band'],ascending = False)
PL_HIGH_RISK_MODEL_payout_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.992-1,76,92,93,102,75,70
e_0.988-0.992,57,85,95,108,107,61
d_0.984-0.988,65,84,87,91,107,59
c_0.977-0.984,67,103,125,127,111,102
b_0-0.977,54,77,87,98,87,77


# DPD 30+

In [65]:
Dpd_PL_HIGH_RISK_MODEL =  PL_HIGH_RISK_MODEL_payout.loc[(PL_HIGH_RISK_MODEL_payout['MONTHS_SINCE_PAYOUT'] == 3) & (PL_HIGH_RISK_MODEL_payout['DPD_EOM'] >= 30)]
Dpd_PL_HIGH_RISK_MODEL_payout = pd.pivot_table(Dpd_PL_HIGH_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_PL_HIGH_RISK_MODEL_payout

band


In [66]:
Dpd_PL_HIGH_RISK_MODEL_payout.columns = [''.join(col).strip() for col in Dpd_PL_HIGH_RISK_MODEL_payout.columns.values]


In [67]:
Dpd_PL_HIGH_RISK_MODEL_payout_x = Dpd_PL_HIGH_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)


In [68]:
divid_Dpd_PL_HIGH_RISK_MODEL_payout_x = Dpd_PL_HIGH_RISK_MODEL_payout_x.div(PL_HIGH_RISK_MODEL_payout_outPut_y.iloc[0])


In [69]:
divid_Dpd_PL_HIGH_RISK_MODEL_payout_y = divid_Dpd_PL_HIGH_RISK_MODEL_payout_x.sort_values(by=['band'],ascending = False)
divid_Dpd_PL_HIGH_RISK_MODEL_payout_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,


## 4.b MED RISK MODEL, 0% POS

In [70]:
PL_MED_RISK= row_data.loc[row_data.CLIENT =='MONEDO_FINANCE_PL']
PL_MED_RISK_MODEL= PL_MED_RISK[(PL_MED_RISK.CUSTOMER_TYPE_DETAILED == '0%_FINANCING - BANK_REJECTED_POS') | (PL_MED_RISK.CUSTOMER_TYPE_DETAILED == '0%_FINANCING_POS')]
bins= [0,0.977,0.983,0.986,0.989,1]
labels = ['b_0-0.977','c_0.977-0.983','d_0.983-0.986','e_0.986-0.989','f_0.989-1']
PL_MED_RISK_MODEL['band'] = pd.cut(PL_MED_RISK_MODEL['SCORE'], bins=bins, labels=labels, right=False)
PL_MED_RISK_MODEL.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)
PL_MED_RISK_MODEL.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
2726,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-23 20:28:27.109000,PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1,0.89192,2020-02-23 20:31:16.074000,1.0,NaN,NaN,2020-02-23 20:31:18.596000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1693895R11,NaN,NaN,ZLP1693895R11,b_0-0.977
2771,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-10 21:49:09.305000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.95565,2020-02-10 21:56:06.150000,1.0,NaN,NaN,2020-02-10 21:56:13.146000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1668381R11,NaN,NaN,ZLP1668381R11,b_0-0.977
2811,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 17:16:32.818000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.93237,2020-02-06 09:53:57.418000,1.0,NaN,NaN,2020-02-06 09:54:19.543000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1658199R11,NaN,NaN,ZLP1658199R11,b_0-0.977
2814,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 15:10:14.386000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.95848,2020-02-05 15:35:45.284000,1.0,NaN,NaN,2020-02-05 15:35:54.704000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1657907R11,NaN,NaN,ZLP1657907R11,b_0-0.977
2826,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-08 14:42:12.625000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.84603,2020-02-08 14:47:14.824000,1.0,NaN,NaN,2020-02-08 14:47:24.564000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING_POS,ZLP1663375R11,NaN,NaN,ZLP1663375R11,b_0-0.977


## LEADD-MED RISK MODEL-0% POS

In [71]:
PL_MED_RISK_MODEL_leadd = PL_MED_RISK_MODEL.loc[(PL_MED_RISK_MODEL['LEADD'] == 1) & (( PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1')|(PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1.0')|(PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0'))]
PL_MED_RISK_MODEL_leadd_outPut= pd.pivot_table(PL_MED_RISK_MODEL_leadd, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_MED_RISK_MODEL_leadd_outPut_x = PL_MED_RISK_MODEL_leadd_outPut.copy()
PL_MED_RISK_MODEL_leadd_outPut_x.columns = [''.join(col).strip() for col in PL_MED_RISK_MODEL_leadd_outPut.columns.values]
PL_MED_RISK_MODEL_leadd_outPut_y= PL_MED_RISK_MODEL_leadd_outPut_x.sort_values(by=['band'],ascending = False)

## OFFER-MED RISK MODEL-0% POS

In [72]:
PL_MED_RISK_MODEL_offer = PL_MED_RISK_MODEL.loc[(PL_MED_RISK_MODEL['OFFER'] == 1) & (( PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1')|(PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1.0')|(PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0'))]
PL_MED_RISK_MODEL_offer_outPut= pd.pivot_table(PL_MED_RISK_MODEL_offer, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_MED_RISK_MODEL_offer_outPut_x = PL_MED_RISK_MODEL_offer_outPut.copy()
PL_MED_RISK_MODEL_offer_outPut_x.columns = [''.join(col).strip() for col in PL_MED_RISK_MODEL_offer_outPut.columns.values]
PL_MED_RISK_MODEL_offer_outPut_y= PL_MED_RISK_MODEL_offer_outPut_x.sort_values(by=['band'],ascending = False)

## payout-MED RISK MODEL-0% POS

In [73]:
PL_MED_RISK_MODEL_payout = PL_MED_RISK_MODEL.loc[(PL_MED_RISK_MODEL['PAYOUT'] == 1) & (( PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1')|(PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1.0')|(PL_MED_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0'))]
PL_MED_RISK_MODEL_payout_outPut= pd.pivot_table(PL_MED_RISK_MODEL_payout, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_MED_RISK_MODEL_payout_outPut_x = PL_MED_RISK_MODEL_payout_outPut.copy()
PL_MED_RISK_MODEL_payout_outPut_x.columns = [''.join(col).strip() for col in PL_MED_RISK_MODEL_payout_outPut.columns.values]
PL_MED_RISK_MODEL_payout_outPut_y= PL_MED_RISK_MODEL_payout_outPut_x.sort_values(by=['band'],ascending = False)


# DPD+ 30

In [74]:
Dpd_PL_MED_RISK_MODEL =  PL_MED_RISK_MODEL_payout.loc[(PL_MED_RISK_MODEL_payout['MONTHS_SINCE_PAYOUT'] == 3) & (PL_MED_RISK_MODEL_payout['DPD_EOM'] >= 30)]
Dpd_PL_MED_RISK_MODEL_payout = pd.pivot_table(Dpd_PL_MED_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_PL_MED_RISK_MODEL_payout.columns = [''.join(col).strip() for col in Dpd_PL_MED_RISK_MODEL_payout.columns.values]
Dpd_PL_MED_RISK_MODEL_payout_x = Dpd_PL_MED_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PL_MED_RISK_MODEL_payout = Dpd_PL_MED_RISK_MODEL_payout_x/PL_MED_RISK_MODEL_payout_outPut_y
divid_Dpd_PL_MED_RISK_MODEL_payout_y = divid_Dpd_PL_MED_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PL_MED_RISK_MODEL_payout_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.989-1,NaN,NaN,NaN,NaN,NaN,NaN
e_0.986-0.989,NaN,NaN,NaN,NaN,NaN,NaN
d_0.983-0.986,0.000000,0.005618,0.000000,0.000000,NaN,NaN
c_0.977-0.983,0.006944,0.000000,0.007092,0.000000,NaN,NaN
b_0-0.977,0.000000,0.013953,0.000000,0.003663,NaN,NaN


# 4.c LOW RISK MODEL-0% POS

In [75]:
PL_MLOW_RISKL= row_data.loc[row_data.CLIENT =='MONEDO_FINANCE_PL']
PL_MLOW_RISK_MODEL= PL_MLOW_RISKL[((PL_MLOW_RISKL.CUSTOMER_TYPE_DETAILED == '0%_FINANCING - BANK_REJECTED_POS') | (PL_MLOW_RISKL.CUSTOMER_TYPE_DETAILED == '0%_FINANCING_POS'))]
bins= [0,0.8950,0.935,0.970,0.985,1]
labels = ['b_0-0.895','c_0.895-0.935','d_0.935-0.970','e_0.970-0.985','f_0.985-1']
PL_MLOW_RISK_MODEL['band'] = pd.cut(PL_MLOW_RISK_MODEL['SCORE'], bins=bins, labels=labels, right=False)
PL_MLOW_RISK_MODEL.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)
PL_MLOW_RISK_MODEL.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
2726,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-23 20:28:27.109000,PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1,0.89192,2020-02-23 20:31:16.074000,1.0,NaN,NaN,2020-02-23 20:31:18.596000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1693895R11,NaN,NaN,ZLP1693895R11,b_0-0.895
2771,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-10 21:49:09.305000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.95565,2020-02-10 21:56:06.150000,1.0,NaN,NaN,2020-02-10 21:56:13.146000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1668381R11,NaN,NaN,ZLP1668381R11,d_0.935-0.970
2811,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 17:16:32.818000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.93237,2020-02-06 09:53:57.418000,1.0,NaN,NaN,2020-02-06 09:54:19.543000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1658199R11,NaN,NaN,ZLP1658199R11,c_0.895-0.935
2814,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 15:10:14.386000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.95848,2020-02-05 15:35:45.284000,1.0,NaN,NaN,2020-02-05 15:35:54.704000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING - BANK_REJECTED_POS,ZLP1657907R11,NaN,NaN,ZLP1657907R11,d_0.935-0.970
2826,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-08 14:42:12.625000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.84603,2020-02-08 14:47:14.824000,1.0,NaN,NaN,2020-02-08 14:47:24.564000,1.0,NaN,NaN,NaN,NaN,0%_FINANCING_POS,ZLP1663375R11,NaN,NaN,ZLP1663375R11,b_0-0.895


## LEADD-LOW RISK MODEL-0% POS

In [76]:
PL_MLOW_RISK_MODEL_leadd = PL_MLOW_RISK_MODEL.loc[(PL_MLOW_RISK_MODEL['LEADD'] == 1) & ( PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')|(PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')]
PL_MLOW_RISK_MODEL_leadd_outPut= pd.pivot_table(PL_MLOW_RISK_MODEL_leadd, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_MLOW_RISK_MODEL_leadd_outPut_x = PL_MLOW_RISK_MODEL_leadd_outPut.copy()
PL_MLOW_RISK_MODEL_leadd_outPut_x.columns = [''.join(col).strip() for col in PL_MLOW_RISK_MODEL_leadd_outPut.columns.values]
PL_MLOW_RISK_MODEL_leadd_outPut_y= PL_MLOW_RISK_MODEL_leadd_outPut_x.sort_values(by=['band'],ascending = False)
PL_MLOW_RISK_MODEL_leadd_outPut_y

,LEADD2020-02,LEADD2020-03,LEADD2020-04,LEADD2020-05,LEADD2020-06,LEADD2020-07
band,,,,,,
f_0.985-1,9,17,19,24,22,19
e_0.970-0.985,19,23,35,52,56,36
d_0.935-0.970,35,43,58,87,72,63
c_0.895-0.935,23,21,37,39,38,32
b_0-0.895,26,48,47,62,46,50


## OFFER-LOW RISK MODEL-0% POS

In [77]:
PL_MLOW_RISK_MODEL_OFFER = PL_MLOW_RISK_MODEL.loc[(PL_MLOW_RISK_MODEL['OFFER'] == 1) & ( PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')|(PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')]
PL_MLOW_RISK_MODEL_OFFER_outPut= pd.pivot_table(PL_MLOW_RISK_MODEL_leadd, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_MLOW_RISK_MODEL_OFFER_outPut_x = PL_MLOW_RISK_MODEL_OFFER_outPut.copy()
PL_MLOW_RISK_MODEL_OFFER_outPut_x.columns = [''.join(col).strip() for col in PL_MLOW_RISK_MODEL_OFFER_outPut.columns.values]
PL_MLOW_RISK_MODEL_OFFER_outPut_y= PL_MLOW_RISK_MODEL_OFFER_outPut_x.sort_values(by=['band'],ascending = False)

## payout-LOW RISK MODEL-0% POS

In [78]:
PL_MLOW_RISK_MODEL_PAYOUT = PL_MLOW_RISK_MODEL.loc[(PL_MLOW_RISK_MODEL['PAYOUT'] == 1) & ( PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')|(PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_MLOW_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')]
PL_MLOW_RISK_MODEL_PAYOUT_outPut= pd.pivot_table(PL_MLOW_RISK_MODEL_leadd, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_MLOW_RISK_MODEL_PAYOUT_outPut_x = PL_MLOW_RISK_MODEL_PAYOUT_outPut.copy()
PL_MLOW_RISK_MODEL_PAYOUT_outPut_x.columns = [''.join(col).strip() for col in PL_MLOW_RISK_MODEL_PAYOUT_outPut.columns.values]
PL_MLOW_RISK_MODEL_PAYOUT_outPut_y= PL_MLOW_RISK_MODEL_PAYOUT_outPut_x.sort_values(by=['band'],ascending = False)
PL_MLOW_RISK_MODEL_PAYOUT_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.985-1,7,14,16,24,21,17
e_0.970-0.985,19,21,34,51,54,30
d_0.935-0.970,34,39,55,86,66,60
c_0.895-0.935,22,20,35,38,38,30
b_0-0.895,8,12,18,19,18,18


# dpd 30+

In [79]:
Dpd_PL_low_RISK_MODEL = PL_MLOW_RISK_MODEL_PAYOUT.loc[(PL_MLOW_RISK_MODEL_PAYOUT['MONTHS_SINCE_PAYOUT'] == 3) & (PL_MLOW_RISK_MODEL_PAYOUT['DPD_EOM'] >= 30)]
Dpd_PL_low_RISK_MODEL

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band


In [80]:
Dpd_PL_low_RISK_MODEL =  PL_MLOW_RISK_MODEL_PAYOUT.loc[(PL_MLOW_RISK_MODEL_PAYOUT['MONTHS_SINCE_PAYOUT'] == 3) & (PL_MLOW_RISK_MODEL_PAYOUT['DPD_EOM'] >= 30)]
Dpd_PL_low_RISK_MODEL_payout = pd.pivot_table(Dpd_PL_low_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_PL_low_RISK_MODEL_payout.columns = [''.join(col).strip() for col in Dpd_PL_low_RISK_MODEL_payout.columns.values]
Dpd_PL_low_RISK_MODEL_payout_x = Dpd_PL_low_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PL_low_RISK_MODEL_payout = Dpd_PL_low_RISK_MODEL_payout_x/PL_MLOW_RISK_MODEL_PAYOUT_outPut_y
divid_Dpd_PL_low_RISK_MODEL_payout_y = divid_Dpd_PL_low_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PL_low_RISK_MODEL_payout_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.985-1,NaN,NaN,NaN,NaN,NaN,NaN
e_0.970-0.985,NaN,NaN,NaN,NaN,NaN,NaN
d_0.935-0.970,NaN,NaN,NaN,NaN,NaN,NaN
c_0.895-0.935,NaN,NaN,NaN,NaN,NaN,NaN
b_0-0.895,NaN,NaN,NaN,NaN,NaN,NaN


# 4. Regular Segments

## 4.a HIGH RISK MODEL

In [81]:
PL_HIGH_RISK= row_data.loc[row_data.CLIENT =='MONEDO_FINANCE_PL']
PL_HIGH_RISK_MODEL= PL_HIGH_RISK[(PL_HIGH_RISK.CUSTOMER_TYPE_DETAILED == 'REGULAR_POS') | (PL_HIGH_RISK.CUSTOMER_TYPE_DETAILED == 'BANK_REJECTED_POS')]
bins= [0,0.963,0.972,0.984,0.990,1]
labels = ['b_0-0.963','c_0.963-0.972','d_0.972-0.984','e_0.984-0.990','f_0.990-1']
PL_HIGH_RISK_MODEL['band'] = pd.cut(PL_HIGH_RISK_MODEL['SCORE'], bins=bins, labels=labels, right=False)
PL_HIGH_RISK_MODEL.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)
PL_HIGH_RISK_MODEL.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
2733,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 16:42:55.579000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.91052,2020-02-05 16:49:03.281000,1.0,NaN,NaN,2020-02-05 16:49:33.243000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1658119R11,NaN,NaN,PLP1658119R11,b_0-0.963
2739,2020-02,MONEDO_FINANCE_PL,RECURRING,2020-02-19 16:21:15.434000,PL-PRED-Score1-mc9c3f0-1,1.00000,2020-02-19 16:21:39.481000,1.0,NaN,NaN,2020-02-19 16:22:24.708000,1.0,NaN,NaN,NaN,NaN,REGULAR_POS,PLP1686207R11,NaN,NaN,PLP1686207R11,NaN
2744,2020-02,MONEDO_FINANCE_PL,RECURRING,2020-02-07 23:17:34.068000,PL-PRED-Score1-mc9c3f0-1,1.00000,2020-02-07 23:18:05.911000,1.0,NaN,NaN,2020-02-07 23:18:17.138000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1662651R11,NaN,NaN,PLP1662651R11,NaN
2750,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-09 21:38:47.055000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.89891,2020-02-09 21:42:53.394000,1.0,NaN,NaN,2020-02-09 21:43:03.173000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1665933R11,NaN,NaN,PLP1665933R11,b_0-0.963
2782,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-15 16:13:25.579000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.53170,2020-02-15 16:22:31.190000,1.0,NaN,NaN,2020-02-15 16:22:39.769000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1677871R11,NaN,NaN,PLP1677871R11,b_0-0.963


## LEADD-HIGH RISK MODEL

In [82]:
PL_LEADD_HIGH_RISK_MODEL = PL_HIGH_RISK_MODEL.loc[(PL_HIGH_RISK_MODEL['LEADD'] == 1) & ( PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1.0')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3')]
PL_LEADD_HIGH_RISK_MODEL_outPut= pd.pivot_table(PL_LEADD_HIGH_RISK_MODEL, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_LEADD_HIGH_RISK_MODEL_outPut_x = PL_LEADD_HIGH_RISK_MODEL_outPut.copy()
PL_LEADD_HIGH_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in PL_LEADD_HIGH_RISK_MODEL_outPut.columns.values]
PL_LEADD_HIGH_RISK_MODEL_outPut_y= PL_LEADD_HIGH_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)

## OFFER-HIGH RISK MODEL

In [83]:
PL_OFFER_HIGH_RISK_MODEL = PL_HIGH_RISK_MODEL.loc[(PL_HIGH_RISK_MODEL['OFFER'] == 1) & ( PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1.0')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3')]
PL_OFFER_HIGH_RISK_MODEL_outPut= pd.pivot_table(PL_LEADD_HIGH_RISK_MODEL, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_OFFER_HIGH_RISK_MODEL_outPut_x = PL_OFFER_HIGH_RISK_MODEL_outPut.copy()
PL_OFFER_HIGH_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in PL_OFFER_HIGH_RISK_MODEL_outPut.columns.values]
PL_OFFER_HIGH_RISK_MODEL_outPut_y= PL_OFFER_HIGH_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)

## payout-HIGH RISK MODEL

In [84]:
PL_PAYOUT_HIGH_RISK_MODEL = PL_HIGH_RISK_MODEL.loc[(PL_HIGH_RISK_MODEL['PAYOUT'] == 1) & ( PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1.0')|(PL_HIGH_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3')]
PL_PAYOUT_HIGH_RISK_MODEL_outPut= pd.pivot_table(PL_LEADD_HIGH_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PL_PAYOUT_HIGH_RISK_MODEL_outPut_x = PL_PAYOUT_HIGH_RISK_MODEL_outPut.copy()
PL_PAYOUT_HIGH_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in PL_PAYOUT_HIGH_RISK_MODEL_outPut.columns.values]
PL_PAYOUT_HIGH_RISK_MODEL_outPut_y= PL_PAYOUT_HIGH_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)


# DPD +30

In [85]:
Dpd_PL_PAYOUT_HIGH_RISK_MODEL =  PL_PAYOUT_HIGH_RISK_MODEL.loc[(PL_PAYOUT_HIGH_RISK_MODEL['MONTHS_SINCE_PAYOUT'] == 3) & (PL_PAYOUT_HIGH_RISK_MODEL['DPD_EOM'] >= 30)]
Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout = pd.pivot_table(Dpd_PL_PAYOUT_HIGH_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout.columns = [''.join(col).strip() for col in Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout.columns.values]
Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout_x = Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout = Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout_x/PL_PAYOUT_HIGH_RISK_MODEL_outPut_y
divid_Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout_y = divid_Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.990-1,NaN,NaN,NaN,NaN,NaN,NaN
e_0.984-0.990,0.017241,0.010101,0.012048,0.000000,NaN,NaN
d_0.972-0.984,0.000000,0.013072,0.010363,0.005208,NaN,NaN
c_0.963-0.972,0.000000,0.000000,0.009901,0.000000,NaN,NaN
b_0-0.963,0.000000,0.017857,0.007246,0.009901,NaN,NaN


# 4.b Med RISK MODEL

In [86]:
PL_Med_RISK= row_data.loc[row_data.CLIENT =='MONEDO_FINANCE_PL']
PL_Med_RISK_MODEL= PL_Med_RISK[(PL_Med_RISK.CUSTOMER_TYPE_DETAILED == 'REGULAR_POS') | (PL_Med_RISK.CUSTOMER_TYPE_DETAILED == 'BANK_REJECTED_POS')]
bins= [0,0.981,0.985,0.987,0.989,1]
labels = ['b_0-0.981','c_0.981-0.985','d_0.985-0.987','e_0.987-0.989','f_0.989-1']
PL_Med_RISK_MODEL['band'] = pd.cut(PL_Med_RISK_MODEL['SCORE'], bins=bins, labels=labels, right=False)
PL_Med_RISK_MODEL.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)
PL_Med_RISK_MODEL.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
2733,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 16:42:55.579000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.91052,2020-02-05 16:49:03.281000,1.0,NaN,NaN,2020-02-05 16:49:33.243000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1658119R11,NaN,NaN,PLP1658119R11,b_0-0.981
2739,2020-02,MONEDO_FINANCE_PL,RECURRING,2020-02-19 16:21:15.434000,PL-PRED-Score1-mc9c3f0-1,1.00000,2020-02-19 16:21:39.481000,1.0,NaN,NaN,2020-02-19 16:22:24.708000,1.0,NaN,NaN,NaN,NaN,REGULAR_POS,PLP1686207R11,NaN,NaN,PLP1686207R11,NaN
2744,2020-02,MONEDO_FINANCE_PL,RECURRING,2020-02-07 23:17:34.068000,PL-PRED-Score1-mc9c3f0-1,1.00000,2020-02-07 23:18:05.911000,1.0,NaN,NaN,2020-02-07 23:18:17.138000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1662651R11,NaN,NaN,PLP1662651R11,NaN
2750,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-09 21:38:47.055000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.89891,2020-02-09 21:42:53.394000,1.0,NaN,NaN,2020-02-09 21:43:03.173000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1665933R11,NaN,NaN,PLP1665933R11,b_0-0.981
2782,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-15 16:13:25.579000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.53170,2020-02-15 16:22:31.190000,1.0,NaN,NaN,2020-02-15 16:22:39.769000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1677871R11,NaN,NaN,PLP1677871R11,b_0-0.981


## LEADD-MED RISK MODEL

In [87]:
LEADD_PL_Med_RISK_MODEL = PL_Med_RISK_MODEL.loc[(PL_Med_RISK_MODEL['LEADD'] == 1) & ( PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1')|(PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1.0')|(PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0')]
LEADD_PL_Med_RISK_MODEL_outPut= pd.pivot_table(LEADD_PL_Med_RISK_MODEL, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
LEADD_PL_Med_RISK_MODEL_outPut_x = LEADD_PL_Med_RISK_MODEL_outPut.copy()
LEADD_PL_Med_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in LEADD_PL_Med_RISK_MODEL_outPut.columns.values]
LEADD_PL_Med_RISK_MODEL_outPut_y= LEADD_PL_Med_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)

# OFFER-MED RISK MODEL

In [88]:
OFFER_PL_Med_RISK_MODEL = PL_Med_RISK_MODEL.loc[(PL_Med_RISK_MODEL['OFFER'] == 1) & ( PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1')|(PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1.0')|(PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0')]
OFFER_PL_Med_RISK_MODEL_outPut= pd.pivot_table(OFFER_PL_Med_RISK_MODEL, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
OFFER_PL_Med_RISK_MODEL_outPut_x = OFFER_PL_Med_RISK_MODEL_outPut.copy()
OFFER_PL_Med_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in OFFER_PL_Med_RISK_MODEL_outPut.columns.values]
OFFER_PL_Med_RISK_MODEL_outPut_y= OFFER_PL_Med_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)

# payout-MED RISK MODEL

In [89]:
PAYOUT_PL_Med_RISK_MODEL = PL_Med_RISK_MODEL.loc[(PL_Med_RISK_MODEL['PAYOUT'] == 1) & ( PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1')|(PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0-1.0')|(PL_Med_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-MED-NEW-PRED-CB_PAYU-MS-m0563c0')]
PAYOUT_PL_Med_RISK_MODEL_outPut= pd.pivot_table(LEADD_PL_Med_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PAYOUT_PL_Med_RISK_MODEL_outPut_x = PAYOUT_PL_Med_RISK_MODEL_outPut.copy()
PAYOUT_PL_Med_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in PAYOUT_PL_Med_RISK_MODEL_outPut.columns.values]
PAYOUT_PL_Med_RISK_MODEL_outPut_y= PAYOUT_PL_Med_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)
PAYOUT_PL_Med_RISK_MODEL_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.989-1,69,95,117,97,91,115
e_0.987-0.989,57,82,125,92,104,115
d_0.985-0.987,63,87,94,91,98,98
c_0.981-0.985,96,130,164,153,147,142
b_0-0.981,75,109,289,175,119,129


# DPD 30+ 

In [90]:
Dpd_PAYOUT_PL_Med_RISK_MODEL =  PAYOUT_PL_Med_RISK_MODEL.loc[(PAYOUT_PL_Med_RISK_MODEL['MONTHS_SINCE_PAYOUT'] == 3) & (PAYOUT_PL_Med_RISK_MODEL['DPD_EOM'] >= 30)]
Dpd_PAYOUT_PL_Med_RISK_MODEL_payout = pd.pivot_table(Dpd_PAYOUT_PL_Med_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_PAYOUT_PL_Med_RISK_MODEL_payout.columns = [''.join(col).strip() for col in Dpd_PAYOUT_PL_Med_RISK_MODEL_payout.columns.values]
Dpd_PAYOUT_PL_Med_RISK_MODEL_payout_x = Dpd_PAYOUT_PL_Med_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout= Dpd_PAYOUT_PL_Med_RISK_MODEL_payout_x.div(PAYOUT_PL_Med_RISK_MODEL_outPut_y)
divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout_y=divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.989-1,NaN,NaN,NaN,NaN,NaN,NaN
e_0.987-0.989,0.000000,0.000000,0.01600,0.000000,NaN,NaN
d_0.985-0.987,NaN,NaN,NaN,NaN,NaN,NaN
c_0.981-0.985,0.000000,0.000000,0.00000,0.006536,NaN,NaN
b_0-0.981,0.013333,0.018349,0.00692,0.000000,NaN,NaN


# 4.c Low RISK MODEL

In [91]:
PL_low_RISK= row_data.loc[row_data.CLIENT =='MONEDO_FINANCE_PL']
PL_low_RISK_MODEL= PL_low_RISK[(PL_low_RISK.CUSTOMER_TYPE_DETAILED == 'REGULAR_POS') | (PL_low_RISK.CUSTOMER_TYPE_DETAILED == 'BANK_REJECTED_POS')]
bins= [0,0.947,0.959,0.972,0.982,1]
labels = ['b_0-0.947','c_0.947-0.959','d_0.959-0.972','e_0.972-0.982','f_0.982-1']
PL_low_RISK_MODEL['band'] = pd.cut(PL_low_RISK_MODEL['SCORE'], bins=bins, labels=labels, right=False)
PL_low_RISK_MODEL.drop_duplicates(subset ="APPLICATION_ID", keep = 'first', inplace = True)
PL_low_RISK_MODEL.head()

,COHORT,CLIENT,NEW_RECURRING,CREATEDAT,SCORING_MODEL,SCORE,LEAD_DATE_EXCL_AUTOREJECTS,LEADD,LAST_OFFER_DATE,OFFER,REJECT_DATE,REJECTD,OFFER_ACCEPTED_DATE,OFFER_ACCEPTED,PAYOUT_DATE,PAYOUT,CUSTOMER_TYPE_DETAILED,APPLICATION_ID,DPD_EOM,MONTHS_SINCE_PAYOUT,UUID,band
2733,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-05 16:42:55.579000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.91052,2020-02-05 16:49:03.281000,1.0,NaN,NaN,2020-02-05 16:49:33.243000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1658119R11,NaN,NaN,PLP1658119R11,b_0-0.947
2739,2020-02,MONEDO_FINANCE_PL,RECURRING,2020-02-19 16:21:15.434000,PL-PRED-Score1-mc9c3f0-1,1.00000,2020-02-19 16:21:39.481000,1.0,NaN,NaN,2020-02-19 16:22:24.708000,1.0,NaN,NaN,NaN,NaN,REGULAR_POS,PLP1686207R11,NaN,NaN,PLP1686207R11,NaN
2744,2020-02,MONEDO_FINANCE_PL,RECURRING,2020-02-07 23:17:34.068000,PL-PRED-Score1-mc9c3f0-1,1.00000,2020-02-07 23:18:05.911000,1.0,NaN,NaN,2020-02-07 23:18:17.138000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1662651R11,NaN,NaN,PLP1662651R11,NaN
2750,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-09 21:38:47.055000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.89891,2020-02-09 21:42:53.394000,1.0,NaN,NaN,2020-02-09 21:43:03.173000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1665933R11,NaN,NaN,PLP1665933R11,b_0-0.947
2782,2020-02,MONEDO_FINANCE_PL,NEW,2020-02-15 16:13:25.579000,PL-POS-HIGH-NEW-PRED-KK_CB_PAYU-MS-me09ab3-1,0.53170,2020-02-15 16:22:31.190000,1.0,NaN,NaN,2020-02-15 16:22:39.769000,1.0,NaN,NaN,NaN,NaN,BANK_REJECTED_POS,PLP1677871R11,NaN,NaN,PLP1677871R11,b_0-0.947


## LEADD-low RISK MODEL

In [92]:
LEADD_low_RISK_MODEL = PL_low_RISK_MODEL.loc[(PL_low_RISK_MODEL['LEADD'] == 1) & ( PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')|(PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')]
LEADD_low_RISK_MODEL_outPut= pd.pivot_table(LEADD_low_RISK_MODEL, values=['LEADD'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
LEADD_low_RISK_MODEL_outPut_x = LEADD_low_RISK_MODEL_outPut.copy()
LEADD_low_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in LEADD_low_RISK_MODEL_outPut.columns.values]
LEADD_low_RISK_MODEL_outPut_y= LEADD_low_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)

## OFFER-low RISK MODEL

In [93]:
OFFER_low_RISK_MODEL = PL_low_RISK_MODEL.loc[(PL_low_RISK_MODEL['OFFER'] == 1) & ( PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')|(PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')]
OFFER_low_RISK_MODEL_outPut= pd.pivot_table(OFFER_low_RISK_MODEL, values=['OFFER'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
OFFER_low_RISK_MODEL_outPut_x = OFFER_low_RISK_MODEL_outPut.copy()
OFFER_low_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in OFFER_low_RISK_MODEL_outPut.columns.values]
OFFER_low_RISK_MODEL_outPut_y= OFFER_low_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)

## PAYOUT-low RISK MODEL

In [94]:
PAYOUT_low_RISK_MODEL = PL_low_RISK_MODEL.loc[(PL_low_RISK_MODEL['PAYOUT'] == 1) & ( PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1')|(PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b-1.0')|(PL_low_RISK_MODEL['SCORING_MODEL'] == 'PL-POS-LOW-NEW-PRED-CB_PAYU-MS-m5fb04b')]
PAYOUT_low_RISK_MODEL_outPut= pd.pivot_table(PAYOUT_low_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins=False ,aggfunc="count")
PAYOUT_low_RISK_MODEL_outPut_x = PAYOUT_low_RISK_MODEL_outPut.copy()
PAYOUT_low_RISK_MODEL_outPut_x.columns = [''.join(col).strip() for col in PAYOUT_low_RISK_MODEL_outPut.columns.values]
PAYOUT_low_RISK_MODEL_outPut_y= PAYOUT_low_RISK_MODEL_outPut_x.sort_values(by=['band'],ascending = False)
PAYOUT_low_RISK_MODEL_outPut_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.982-1,13,14,12,19,12,16
e_0.972-0.982,7,10,12,16,23,12
d_0.959-0.972,17,13,14,21,26,20
c_0.947-0.959,9,11,13,15,12,16
b_0-0.947,11,17,21,24,17,12


# DPD 30+

In [95]:
Dpd_PAYOUT_PL_low_RISK_MODEL =  PAYOUT_low_RISK_MODEL.loc[(PAYOUT_low_RISK_MODEL['MONTHS_SINCE_PAYOUT'] == 3) & (PAYOUT_low_RISK_MODEL['DPD_EOM'] >= 30)]


In [96]:
Dpd_PAYOUT_PL_low_RISK_MODEL =  PAYOUT_low_RISK_MODEL.loc[(PAYOUT_low_RISK_MODEL['MONTHS_SINCE_PAYOUT'] == 3) & (PAYOUT_low_RISK_MODEL['DPD_EOM'] >= 30)]
Dpd_PAYOUT_PL_low_RISK_MODEL_payout = pd.pivot_table(Dpd_PAYOUT_PL_low_RISK_MODEL, values=['PAYOUT'], index=['band'],columns=['COHORT'],fill_value=0,margins= False ,aggfunc="count")
Dpd_PAYOUT_PL_low_RISK_MODEL_payout.columns = [''.join(col).strip() for col in Dpd_PAYOUT_PL_low_RISK_MODEL_payout.columns.values]
Dpd_PAYOUT_PL_low_RISK_MODEL_payout_x = Dpd_PAYOUT_PL_low_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout = Dpd_PAYOUT_PL_low_RISK_MODEL_payout_x.div(PAYOUT_low_RISK_MODEL_outPut_y)
divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout_y = divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout.sort_values(by=['band'],ascending = False)
divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout_y

,PAYOUT2020-02,PAYOUT2020-03,PAYOUT2020-04,PAYOUT2020-05,PAYOUT2020-06,PAYOUT2020-07
band,,,,,,
f_0.982-1,NaN,NaN,NaN,NaN,NaN,NaN
e_0.972-0.982,NaN,NaN,NaN,NaN,NaN,NaN
d_0.959-0.972,NaN,NaN,NaN,NaN,NaN,NaN
c_0.947-0.959,NaN,NaN,NaN,NaN,NaN,NaN
b_0-0.947,NaN,NaN,NaN,NaN,NaN,NaN


# Poland_excel_export

In [97]:
path = r"C:\Users\abdalla.khiralla\Model_score_outPut.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

Lead_PL_IL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=1,header=True)
offer_PL_IL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=10,header=True)
payout_PL_IL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=19,header=True)
PL_IL_Rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=1,header=True)
offer_PL_IL_Rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=10,header=True)
payout_PL_IL_Rec_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=19,header=True)
PL_BNPL_leads_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=82,header=True)
PL_OFFER_BNPL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=91,header=True)
PL_BNPL_PAYOUT_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=100,header=True)
PL_HIGH_RISK_MODEL_leadd_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=28,header=True)
PL_HIGH_RISK_MODEL_OFFER_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=37,header=True)
PL_HIGH_RISK_MODEL_payout_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=46,header=True)
PL_MED_RISK_MODEL_leadd_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=28,header=True)
PL_HIGH_RISK_MODEL_OFFER_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=37,header=True)
PL_MED_RISK_MODEL_payout_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=46,header=True)
PL_MLOW_RISK_MODEL_leadd_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=28,header=True)
PL_MLOW_RISK_MODEL_OFFER_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=37,header=True)
PL_MLOW_RISK_MODEL_PAYOUT_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=46,header=True)
PL_LEADD_HIGH_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=55,header=True)
PL_OFFER_HIGH_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=64,header=True)
PL_PAYOUT_HIGH_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=73,header=True)
LEADD_PL_Med_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=55,header=True)
OFFER_PL_Med_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=64,header=True)
PAYOUT_PL_Med_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=48,startcol=73,header=True)
LEADD_low_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=55,header=True)
OFFER_low_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=64,header=True)
PAYOUT_low_RISK_MODEL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=6,startcol=73,header=True)
divid_Dpd_IL_new_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=31,startcol=19,header=True)
divid_Dpd_IL_rec_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=73,startcol=19,header=True)
divid_Dpd_BNPL_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=31,startcol=100,header=True)
divid_Dpd_PL_HIGH_RISK_MODEL_payout_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=116,startcol=46,header=True)
divid_Dpd_PL_MED_RISK_MODEL_payout_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=73,startcol=46,header=True)
divid_Dpd_PL_low_RISK_MODEL_payout_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=31,startcol=46,header=True)
divid_Dpd_PL_PAYOUT_HIGH_RISK_MODEL_payout_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=116,startcol=73,header=True)
divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=73,startcol=73,header=True)
divid_Dpd_PAYOUT_PL_Med_RISK_MODEL_payout_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=31,startcol=73,header=True)
divid_Dpd_IL_rec_cross_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=31,startcol=73,header=True)
PL_IL_Rec_cross_outPut.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=1,header=True)
offer_PL_IL_Rec_outPut_cross_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=10,header=True)
payout_PL_IL_Rec_cross_outPut_y.to_excel(writer, sheet_name = 'Model_Score_Distribution_PL',startrow=91,startcol=19,header=True)

writer.save()
writer.close()